## ML-第一版本-类别特征编码
该版本只用原始参数进行建模，得到最最初始结果

### 结果记录
+ OrdinalEncoder: 训练集AUC均值：0.7446909721024187, 训练集合AUC方差: 0.0055409887547691675
+ Count Encoder: 训练集AUC均值：0.746436171821856, 训练集合AUC方差: 0.003175456365187765
+ OneHotEncoder: 训练集AUC均值：0.7265152230144775, 训练集合AUC方差: 0.004283383774893741
+ Summary Encoder: 训练集AUC均值：0.7229403477581448, 训练集合AUC方差: 0.005148684305762446
+ Backward Difference Encoder: 训练集AUC均值：0.7432279154831029, 训练集合AUC方差: 0.0061828342928654245
+ Helmert Encoder: 训练集AUC均值：0.7435028474177601, 训练集合AUC方差: 0.005320239374439747
+ Polynomial Encoder: 训练集AUC均值：0.664800290588822, 训练集合AUC方差: 0.005266656000262627
+ Binary Encoder: 训练集AUC均值：0.7439132481659769, 训练集合AUC方差: 0.0053505487194486865
+ BaseN Encoder: 训练集AUC均值：0.7439132481659769, 训练集合AUC方差: 0.0053505487194486865
+ Hashing Encoder: 训练集AUC均值：0.6699049732137634, 训练集合AUC方差: 0.0013664330952263192
+ Target Encoder: 训练集AUC均值：0.7542066688653313, 训练集合AUC方差: 0.006100909854860348
+ MEstimateEncoder: 训练集AUC均值：0.7565341153554674, 训练集合AUC方差: 0.005970607753749473
+ JamesStein Encoder: 训练集AUC均值：0.7568955097744434, 训练集合AUC方差: 0.006076080528502473
+ Generalized Linear Mixed Model Encoder:
+ WOE Encoder: 训练集AUC均值：0.7546565408290457, 训练集合AUC方差: 0.006299987226037399
+ Leave One Out Encoder: 训练集AUC均值：1.0, 训练集合AUC方差: 0.0
+ CatBoost Encoder: 训练集AUC均值：0.9408993759272513, 训练集合AUC方差: 0.003510999287948365

### 总结
1. 考虑过Leave One Out Encoder和CatBoost Encoder两个得到的结果，auc提交上去结果为0.5，该结果不可靠
2. 从以上所有内容中选woe encoder作为baseline，原因是自己比较了解woe编码，方法相对稳定

## 导包

In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns


import scorpyo as sp
from null_importance import get_null_importance
from time_sequence_feats import get_time_base, get_sequence_statis, get_sequence_groupby_statis
import category_encoders as ce

from gensim.models import word2vec



import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_rows', 320, 'max_columns',100)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_version  = os.path.join(path_results, 'version_1')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


path_new_train = os.path.join(path_new_data, 'train_lightgbm_20221014.csv')
path_new_test  = os.path.join(path_new_data, 'test_lightgbm_20221014.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"

In [3]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

## 特征编码

In [4]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk
0,0,guojianping9672,rd,GVhZtW4i1,rqRxAjAL1RYC,firefox_78,firefox,win,win10,2022-01-18 19:10:41,内网,200,成都,2umVQwhiiwNJ,xxx.com/mail,2022-01,0
1,1,yangtao1740,sales,l3MuTMPoQ,iKPTa3su50y7,chrome_93,chrome,win,win11,2022-04-01 17:04:00,内网,200,深圳,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0
2,2,wangying9098,rd,4uHWcskWv,1baNbqxMWcCu,ie_11,ie,win,win10,2022-03-01 15:53:49,内网,200,成都,dwS3cdn15GK4,wpsdoc.xxx.com/kdocs,2022-03,0
3,3,liguixiang3860,rd,mQh3NwtY7,C04Llg4lKl4C,edge_93,edge,win,win10,2022-02-07 19:46:25,内网,200,北京,nHrKgKdJ1Mzt,xxx.com/github,2022-02,0
4,4,guanyu9205,sales,C2QtgDKAZ,kSscjiRSz1aD,edge_93,edge,win,win10,2022-04-12 10:05:19,内网,200,成都,RwHe8Q1R7AlB,business.xxx.com/,2022-04,0


### OrdinalEncoder

In [5]:

# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.OrdinalEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Count Encoder

In [6]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.CountEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### OneHotEncoder

In [7]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.OneHotEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Summary Encoder

In [8]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.SummaryEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Backward Difference Encoder

In [9]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.BackwardDifferenceEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Helmert Encoder

In [10]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.HelmertEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Polynomial Encoder

In [11]:

# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.PolynomialEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Binary Encoder

In [12]:

# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.BinaryEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### BaseN Encoder

In [13]:

# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.BaseNEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Hashing Encoder

In [14]:

# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.HashingEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Target Encoder

In [15]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.TargetEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### MEstimate Encoder

In [16]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.MEstimateEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### JamesStein Encoder

In [17]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.JamesSteinEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### Generalized Linear Mixed Model Encoder

In [18]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.GLMMEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### WOE Encoder

In [19]:
remove_cols = ['id', 'op_datetime', 'op_month']
cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
       'browser_version', 'browser', 'os_type', 'os_version',
       'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


ord_encoder = ce.WOEEncoder(cols=cate_cols)

df_row_val[y_label] = np.nan
df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
df_row_val  = ord_encoder.transform(df_row_val)


display(df_row_train.head())
display(df_row_val.head())

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk
0,0,0.399244,-0.041851,0.123852,0.355653,0.014496,0.014496,0.009044,0.012388,2022-01-18 19:10:41,-0.011044,-0.298867,-0.100602,-0.246014,-0.246014,2022-01,0
1,1,0.090044,0.035442,-0.076528,0.012419,0.060103,-0.009554,0.009044,-0.021990,2022-04-01 17:04:00,-0.011044,-0.298867,-0.069165,-0.150937,-0.150937,2022-04,0
2,2,0.142991,-0.041851,-0.098600,0.092439,-0.059518,-0.068291,0.009044,0.012388,2022-03-01 15:53:49,-0.011044,-0.298867,-0.100602,0.530533,-0.206373,2022-03,0
3,3,-0.011160,-0.041851,-0.255592,-0.085693,0.041217,0.041217,0.009044,0.012388,2022-02-07 19:46:25,-0.011044,-0.298867,0.034066,-0.284901,-0.284901,2022-02,0
4,4,0.232119,0.035442,-0.032103,0.145578,0.041217,0.041217,0.009044,0.012388,2022-04-12 10:05:19,-0.011044,-0.298867,-0.100602,-0.150937,-0.150937,2022-04,0


,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk
0,0,-0.064648,-0.041851,-0.276959,-0.118715,-0.089269,-0.089269,-0.089269,-0.089269,2022-05-25 16:07:00,-0.011044,-0.298867,-0.106403,-0.284901,-0.284901,2022-05,NaN
1,1,0.065465,-0.041851,-0.100239,0.002945,0.041217,0.041217,0.009044,0.012388,2022-06-23 14:48:39,-0.011044,-0.298867,-0.106403,-0.246014,-0.246014,2022-06,NaN
2,2,-0.150053,-0.041851,-0.412955,-0.237064,-0.077409,-0.068291,0.009044,0.012388,2022-06-17 17:22:54,-0.011044,-0.298867,0.034066,-0.284901,-0.284901,2022-06,NaN
3,3,-0.208282,-0.041851,-0.406171,-0.222533,0.041217,0.041217,0.009044,0.012388,2022-05-27 10:13:52,-0.011044,-0.298867,-0.100602,-0.255350,-0.255350,2022-05,NaN
4,4,0.100432,-0.041851,-0.089002,0.072266,0.006565,-0.009554,0.009044,0.012388,2022-05-18 10:47:23,-0.011044,-0.298867,-0.100602,-0.284901,-0.284901,2022-05,NaN


### Leave One Out Encoder

In [20]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.LeaveOneOutEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)

# display(df_row_train.head())
# display(df_row_val.head())

### CatBoost Encoder

In [21]:
# remove_cols = ['id', 'op_datetime', 'op_month']
# cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
#        'browser_version', 'browser', 'os_type', 'os_version',
#        'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


# ord_encoder = ce.CatBoostEncoder(cols=cate_cols)

# df_row_val[y_label] = np.nan
# df_row_train = ord_encoder.fit_transform(df_row_train, df_row_train[y_label] )
# df_row_val  = ord_encoder.transform(df_row_val)


# display(df_row_train.head())
# display(df_row_val.head())

### 获取建模使用的特征

In [22]:
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [23]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

## Modeling

### lightgbm

In [24]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [25]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)


----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.764415	valid_1's auc: 0.756393
[20]	training's auc: 0.768503	valid_1's auc: 0.755391
[30]	training's auc: 0.770732	valid_1's auc: 0.754873
[40]	training's auc: 0.772602	valid_1's auc: 0.754283
[50]	training's auc: 0.774953	valid_1's auc: 0.753952
[60]	training's auc: 0.775819	valid_1's auc: 0.753835
[70]	training's auc: 0.777138	valid_1's auc: 0.753028
[80]	training's auc: 0.779112	valid_1's auc: 0.753024
[90]	training's auc: 0.780647	valid_1's auc: 0.752382
[100]	training's auc: 0.782445	valid_1's auc: 0.751473
Early stopping, best iteration is:
[4]	training's auc: 0.761858	valid_1's auc: 0.759193
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.770557	valid_1's auc: 0.742834
[20]	training's auc: 0.773054	valid_1's auc: 0.742082
[30]	training's auc: 0.776569	valid_1's auc: 0.74178
[40]	training's auc: 0.778908	valid_1's auc: 0.741463
[50]

,name,importance
2,ip_transform,16158.975797
9,http_status_code,11083.731878
12,url,2684.825643
8,ip_type,332.164749
0,user_name,295.542932
3,device_num_transform,138.568509
10,op_city,64.756882
4,browser_version,25.030074
1,department,22.467580
11,log_system_transform,20.071158


训练集AUC均值：0.7546565408290457, 训练集合AUC方差: 0.006299987226037399


In [26]:
# 训练集得分效果
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

训练集AUC均值：0.7546565408290457, 训练集合AUC方差: 0.0


In [27]:
if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)